In [8]:
%load_ext autoreload
%autoreload 2

100%|██████████| 986/986 [00:00<00:00, 6171.38it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from main import *

In [15]:
print(*config.items(), sep='\n')

('model_name', 'LSTM_3')
('feature', 'clean-data extended-length origins')
('max_len', 48)
('min_freq_src', 5)
('min_freq_trg', 5)
('max_freq_sub', 8)
('bpe_vocab_size', 25000)
('embedding_dim', 192)
('hidden_size', 384)
('num_layers', 3)
('num_epochs', 18)
('weight_decay', 0.0001)
('label_smoothing', 0.2)
('dropout_emb', 0.15)
('dropout_enc', 0.2)
('dropout_dec', 0.2)
('dropout_attention', 0.1)
('learning_rate', 0.001)
('lr_manual_decrease', False)
('amsgrad', False)
('gamma', 0.5)
('patience', 1)
('threshold', 0.001)
('batch_size', 128)
('use_tf', False)
('tf_from_epoch', 0)
('tf_start', 0.9)
('tf_decrease', 0.02)
('src_vocab_size', 24367)
('trg_vocab_size', 18099)


In [16]:
from dataset2 import TrainDataLoader, TestDataLoader
from config import filenames, folders
from lstm3 import LSTM_3

In [20]:
unk_idx, pad_idx, bos_idx, eos_idx = 0, 1, 2, 3
device = 'cuda'

train_loader = TrainDataLoader(train_dataset, shuffle=True)
val_loader = TestDataLoader(val_dataset, shuffle=False)

weights_filename = folders['weights'] + 'saves/lstm-save-14.pt'

model = LSTM_3(config=config, weights_filename=weights_filename).to(device)
print(model)
print(model.count_parameters())

LSTM_3(
  (src_embedding): Embedding(24367, 192, padding_idx=1)
  (trg_embedding): Embedding(18099, 192, padding_idx=1)
  (emb_dropout): Dropout(p=0.15, inplace=False)
  (enc_dropout): Dropout(p=0.2, inplace=False)
  (dec_dropout): Dropout(p=0.2, inplace=False)
  (attention_dropout): Dropout(p=0.1, inplace=False)
  (encoder): LSTM(192, 384, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (encoder_output_proj): Linear(in_features=768, out_features=384, bias=True)
  (decoder): LSTM(192, 384, num_layers=3, batch_first=True, dropout=0.2)
  (encoder_hidden_proj): ModuleList(
    (0-2): 3 x Linear(in_features=768, out_features=384, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-2): 3 x Linear(in_features=768, out_features=384, bias=True)
  )
  (fc): Linear(in_features=768, out_features=18099, bias=True)
  (emb_layer_norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  (encoder_output_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  (decode

In [26]:
model.demonstrate(val_dataset, vocab_src, vocab_trg, examples=1, wait=0, verbose=1, beam_width=5)

k: 5	B: 1	max_len: 17
beam tokens:
<BOS>|und  |das  |müssen|wir  |umsetz|,    |bevor|wir  |es   |tun  |.    |<EOS>
index in batch: 0
0.0018|0.0005|0.0004|0.0003|2.0558

	beam probability: 0.18
<BOS>|and  |that |'s   |going|to   |do   |that |before|we   |do   |it   |.    |<EOS>|<PAD>|<PAD>|<PAD>|<PAD>
100.0|74.08|997.42|54.88|11.08|49.66|4.5  |10.03|3.69 |9.07 |1.11 |5.5  |0.74 |4.08 |0.74 |1.66 |0.27 |0.67 
	beam probability: 0.05
<BOS>|and  |that |'s   |going|to   |do   |it   |before|we   |do   |it   |.    |<EOS>|<PAD>|<PAD>|<PAD>|<PAD>
100.0|74.08|997.42|54.88|11.08|49.66|4.5  |0.1  |149.18|0.09 |49.66|0.02 |36.79|0.02 |36.79|0.01 |14.96|0.0  
	beam probability: 0.05
<BOS>|and  |that |'s   |going|to   |do   |this |before|we   |do   |it   |.    |<EOS>|<PAD>|<PAD>|<PAD>|<PAD>
100.0|74.08|997.42|54.88|11.08|49.66|4.5  |0.3  |27.25|0.25 |7.43 |0.14 |6.72 |0.11 |6.72 |0.04 |2.73 |0.02 
	beam probability: 0.03
<BOS>|and  |that |'s   |going|to   |do   |that |before|we   |do   |this |.    |<

In [25]:
train_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['train_src'], 
                                filenames['train_trg'], 
                                max_len=config['max_len'], 
                                device=device,
                                sort_lengths=True)

val_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['test_src'], 
                                filenames['test_trg'], 
                                max_len=72, 
                                device=device, 
                                sort_lengths=True)

100%|██████████| 986/986 [00:00<00:00, 5824.98it/s]


In [14]:
# train_dataset.ids

In [19]:
from submission import get_bleu
from dataset2 import RawDataset
raw_dataset_val = RawDataset(filenames['test_trg'])
raw_dataset_val.src = [raw_dataset_val.src[i] for i in val_dataset.ids]
bleu, sinergy, bp = get_bleu(model, val_dataset, raw_dataset_val, vocab_trg, vocab_src, n=int(1e3), use_beam=False, beam_width=1, border=0.001/100)
print(f'BLEU4: {bleu:0.2f}')

1000


0it [00:00, ?it/s]c:\Users\Dubinin Daniil\Desktop\bhw2\src\submission.py:20: RuntimeWarning: divide by zero encountered in log
  log_sum = np.sum(np.log(a))
<string>:36: RuntimeWarning: invalid value encountered in divide
8it [00:06,  1.21it/s]

correct:	 3 0 0 0
total:		 2250 1264 599 253
sinergy:	0.06
BP:		0.0
0.1/0.0/0.0/0.0
BLEU4: 0.00


In [10]:
from submission import make_submission
from dataset2 import SubmissionDataLoader, SubmissionDataset, RawDataset
raw_dataset = RawDataset(filenames['submission_src'])
submission_dataset = SubmissionDataset(filenames['submission_src'], vocab=vocab_src)
submission_loader = SubmissionDataLoader(submission_dataset)
make_submission(model, submission_loader=submission_loader, raw_dataset=raw_dataset, vocab_trg=vocab_trg, vocab_src=vocab_src, filenames=filenames)

100%|██████████| 24/24 [00:39<00:00,  1.65s/it]


In [95]:
src = "also , als fotografen wissen sie , wenn sie ein foto mit kurzer <UNK> machen , haben sie sehr wenig licht , aber wir <UNK> eine milliarde mal schneller als ihre kürzeste <UNK> , also bekommen sie so gut wie gar kein licht ."
model.translate(src, vocab_src, vocab_trg, beams=5, verbose=1, border=0.00)

k: 5	B: 1	max_len: 48
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cud

"so , as you know , if you know a photograph with a short amount of light , they 're very little light , but we 're going to make a billion times faster than your pure counterparts , so"

In [88]:
len(src.split())

44

In [94]:
import torch
import numpy as np
a = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device='cpu')
b = torch.tensor([1, 2], dtype=torch.long)
a.masked_fill((a>0), 0)
a

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [119]:
val_loader = TestDataLoader(val_dataset, batch_size=512, shuffle=True)


In [37]:
model.demonstrate(val_loader, vocab_src, vocab_trg, examples=1, wait=0, beam_width=1, verbose=2)

k: 1	B: 1	max_len: 27

step:0
beam tokens:
<BOS>|▁als |▁ich |▁11  |▁jahre|▁alt |▁war |▁,   |▁wurde|▁ich |▁eines|▁morge|▁von |▁den |▁kl  |ängen|▁helle|▁freud|▁gewec|▁.   |<EOS>
index in batch: 0
100.0

	beam probability: 10000.00
<BOS>
100.0


step:1
beam tokens:
<BOS>|▁als |▁ich |▁11  |▁jahre|▁alt |▁war |▁,   |▁wurde|▁ich |▁eines|▁morge|▁von |▁den |▁kl  |ängen|▁helle|▁freud|▁gewec|▁.   |<EOS>
index in batch: 0
44.932

	beam probability: 4493.29
<BOS>|when 
100.0|44.93


step:2
beam tokens:
<BOS>|▁als |▁ich |▁11  |▁jahre|▁alt |▁war |▁,   |▁wurde|▁ich |▁eines|▁morge|▁von |▁den |▁kl  |ängen|▁helle|▁freud|▁gewec|▁.   |<EOS>
index in batch: 0
36.787

	beam probability: 3678.79
<BOS>|when |i    
100.0|44.93|81.87


step:3
beam tokens:
<BOS>|▁als |▁ich |▁11  |▁jahre|▁alt |▁war |▁,   |▁wurde|▁ich |▁eines|▁morge|▁von |▁den |▁kl  |ängen|▁helle|▁freud|▁gewec|▁.   |<EOS>
index in batch: 0
24.659

	beam probability: 2465.97
<BOS>|when |i    |was  
100.0|44.93|81.87|30.12


step:4
beam tokens:
<BOS>

In [ ]:
5e3

5000.0